# Terminal

JupyterLab (and many other applications) use [xtermjs](https://github.com/xtermjs/xterm.js) for rendering line-based terminal outputs. 

In [ ]:
import IPython, tornado.ioloop, json
from wxyz.lab.widget_term import Terminal, W, T
from wxyz.lab.widget_dock import DockBox, DockPop
from wxyz.html.widget_color import AlphaColorPicker

In [ ]:
def make_a_terminal_demo(rows=1):
    terminal = Terminal(local_echo=True, rows=rows, fit=False)
    terminal.observe(lambda change: terminal.send({"content": "hello!"}), "active_terminals")
    return terminal

if __name__ == "__main__":
    terminal = make_a_terminal_demo()
    display(terminal)

In [ ]:
def make_a_fancy_terminal_demo():
    t0, t1 = t = [Terminal(description=f"Terminal {i}", fit=True) for i in range(2)]
    
    i = 0
    
    def send(txt, prefix=""):
        [t1.send_line(f"{prefix}{line}") for line in txt.splitlines()]

    t0.on_data(lambda t0, data: send(str(data["content"]), "> "))

    def t0_change(change):
        c = i % 5 + 30
        send(str(change.new), f"\x1B[1;3;{c}m{change.name}\x1B[0m\t")
    
    t0.observe(t0_change, ("scroll", "selection", "rows", "cols", "active_terminals"))

    checks = {it: W.Checkbox(description=it) for it in ["local_echo", "fit"]}
    sliders = {it: W.IntSlider(description=it) for it in ["scroll", "rows", "cols"]}
    
    [T.link((t0, k), (v, "value")) for k, v in sliders.items()]
    [T.link((t0, k), (v, "value")) for k, v in checks.items()]
    [T.dlink((t0, "fit"), (sliders[v], "disabled")) for v in ["rows", "cols"]]
    
    colors = {k: AlphaColorPicker(value=v, description=k) for k, v in t0.theme.items()}
    options =  W.VBox([*colors.values(), *checks.values(), *sliders.values()])
    options.add_traits(description=T.Unicode("Options").tag(sync=True))
    
    def themed(*kwargs):
        new_theme = {k: w.value for k, w in colors.items()}
        t0.theme = new_theme

    [c.observe(themed, "value") for c in colors.values()];

    box = DockBox([*t, options], layout=dict(height="400px"))

    def hi():
        nonlocal i
        i += 1
        c = i % 5 + 30
        t[0].send_line(f"👋  {i} Hello from \x1B[1;3;{c}mxterm.js\x1B[0m")
    hi()

    pcb = tornado.ioloop.PeriodicCallback(hi, 2000)

    pcb.start()
    
    return box

In [ ]:
if __name__ == "__main__":
    box = make_a_fancy_terminal_demo()
    display(box)